In [ ]:
%tensorflow_version 1.x
import keras
import pandas as pd
import numpy as np
import pickle
import time
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, UpSampling1D, RepeatVector,TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Model
from keras.layers import Input, GaussianNoise
from keras.utils import plot_model
from keras.layers.merge import concatenate
from numpy.random import seed
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, LeakyReLU
from keras.models import Model
from keras.layers import Dense, Conv1D, Input, MaxPooling1D, Flatten, Dropout, LSTM
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
#loading dataset and separating training and testing
with open('/content/drive/My Drive/Colab Notebooks/dataset/train', 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        data_train = dict['data']
        label_train = dict['label']
with open('/content/drive/My Drive/Colab Notebooks/dataset/test', 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        data_test = dict['data']
        label_test = dict['label']


In [ ]:
#Normalizing the training data

min_max_scaler = preprocessing.MinMaxScaler()
normalize = min_max_scaler.fit_transform(data_train)



In [ ]:
#Normalizing testing data

min_max_scaler = preprocessing.MinMaxScaler()

normalize_test = min_max_scaler.fit_transform(data_test)
x_test = normalize_test.reshape(normalize_test.shape[0],normalize_test.shape[1],1)

In [ ]:
#shape for passing training data to autoencoder
train_X = normalize.reshape(normalize.shape[0],normalize.shape[1])

In [ ]:
#performing one hot encoding on testing and training labels

y_train = keras.utils.to_categorical(label_train,3)
y_test = keras.utils.to_categorical(label_test,3)



In [ ]:
#dense autoencoder

input_img = Input(shape=(310,))
encoded = Dense(256, activation='relu')(input_img)
encoded = Dense(128, activation='relu')(encoded)
#encoded = Dense(64, activation='relu')(encoded)

#decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(encoded)
decoded = Dense(310, activation='sigmoid')(decoded)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
#compiling the autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
t1 = time.time()
print("time",t1-time.time())

time -3.719329833984375e-05


In [ ]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 310)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               79616     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_4 (Dense)              (None, 310)               79670     
Total params: 225,206
Trainable params: 225,206
Non-trainable params: 0
_________________________________________________________________


In [ ]:
autoencoder.fit(train_X, train_X,
                epochs=50,
                batch_size=64,
                shuffle=False)
t1 = time.time()
print("time",t1-time.time())


Epoch 1/50
84420/84420 [==============================] - 6s 70us/step - loss: 0.0239 - accuracy: 0.2165
Epoch 2/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0238 - accuracy: 0.2460
Epoch 3/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0250 - accuracy: 0.2308
Epoch 4/50
84420/84420 [==============================] - 4s 51us/step - loss: 0.0235 - accuracy: 0.2428
Epoch 5/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0225 - accuracy: 0.2561
Epoch 6/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0211 - accuracy: 0.2559
Epoch 7/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0195 - accuracy: 0.2392
Epoch 8/50
84420/84420 [==============================] - 4s 49us/step - loss: 0.0190 - accuracy: 0.2432
Epoch 9/50
84420/84420 [==============================] - 4s 50us/step - loss: 0.0186 - accuracy: 0.2539
Epoch 10/50
84420/84420 [=============================

In [ ]:
#predicting the train data from autoencoder
predicted_train = autoencoder.predict(train_X)

In [ ]:
#shape of predicted data
predicted_train.shape

(84420, 310)

In [ ]:
#Combining the orginal data and autoencoder output
X_train = np.concatenate((train_X,predicted_train),axis=0)
Y_train = np.concatenate((y_train,y_train),axis=0)

In [ ]:
#checking the shape
X_train.shape

(168840, 310)

In [ ]:
#Reshaping the new train data to pass the combined model
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)

In [ ]:
#shape of new Y-train data
Y_train.shape

(168840, 3)

In [ ]:
#creating a model
visible = Input(shape=(310,1))

In [ ]:
#creating the first model 
model1 = GaussianNoise(0.05)(visible)
model1 = Conv1D(filters=64, kernel_size=2, activation='relu')(model1)
model1 = MaxPooling1D(pool_size=1)(model1)
model1 = Dropout(0.5)(model1)
model1 = Conv1D(filters=128, kernel_size=3, activation='relu')(model1)
model1 = MaxPooling1D(pool_size=1)(model1)
model1 = Dropout(0.5)(model1)
model1 = Conv1D(filters=256, kernel_size=3, activation='relu')(model1)
model1 = MaxPooling1D(pool_size=1)(model1)
model1 = Dropout(0.5)(model1)
model1 = Flatten()(model1)



In [ ]:
#Creating second model
model2 = GaussianNoise(0.05)(visible)
model2 = Conv1D(64, 2, padding='same')(model2)
model2 = LeakyReLU(alpha=0.05)(model2)
model2 = LSTM(64, return_sequences=True)(model2)
model2 = LeakyReLU(alpha=0.05)(model2)
model2 = MaxPooling1D(1)(model2)
model2 = Conv1D(128, 3, padding='same')(model2)
model2 = LeakyReLU(alpha=0.05)(model2)
model2 = LSTM(128, return_sequences=True)(model2)
model2 = LeakyReLU(alpha=0.05)(model2)
model2 = MaxPooling1D(1)(model2)
model2 = Conv1D(256, 3, padding='same')(model2)
model2 = LeakyReLU(alpha=0.05)(model2)
model2 = LSTM(256)(model2)
model2 = LeakyReLU(alpha=0.05)(model2)

In [ ]:
#combining the model
merge = concatenate([model1,model2])

In [ ]:
hidden1 = Dense(400, activation='relu')(merge)
drop1 = Dropout(0.5)(hidden1)
hidden2 = Dense(150, activation='relu')(drop1)
drop2 = Dropout(0.5)(hidden2)
output = Dense(3, activation='softmax')(hidden2)
combined_model = Model(inputs=visible, outputs=output)

In [ ]:
combined_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 310, 1)       0                                            
__________________________________________________________________________________________________
gaussian_noise_2 (GaussianNoise (None, 310, 1)       0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 310, 64)      192         gaussian_noise_2[0][0]           
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 310, 64)      0           conv1d_4[0][0]                   
____________________________________________________________________________________________

In [ ]:
#compiling the model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
t = time.time()
print("time",t-time.time())
time_callback = TimeHistory()
history = combined_model.fit(X_train,Y_train,epochs=10,shuffle=True,validation_data=(x_test,y_test),batch_size=128,callbacks=[time_callback])
t = time.time()
print("time",t-time.time())

time -4.291534423828125e-05
Train on 168840 samples, validate on 58128 samples
Epoch 1/10
 73600/168840 [============>.................] - ETA: 10:48 - loss: 0.9635 - accuracy: 0.4830

In [ ]:
#total computing time for every epoch
times = time_callback.times

In [ ]:
print("computing time in every epoch",times)

In [ ]:
#evaluating the model
combined_model.evaluate(x_test,y_test,batch_size=128,verbose=1)

In [ ]:
#saving the model
combined_model.save('/content/drive/My Drive/Colab Notebooks/DL_combined_model.h5')

In [ ]:
#graph
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()